In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/

Mounted at /content/gdrive
/content/gdrive/My Drive


In [ ]:
import pandas as pd
data = pd.read_excel(r'Time series project/Data/ATM/NN5.xlsx')

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

# Set 'Date' as the index for easy time series manipulation
data.set_index('Date', inplace=True)
data.replace(0, pd.NA, inplace=True)  # Replace zeros with NaN
data.fillna(data.mean(), inplace=True)  # Fill NaN with the column mean

<ipython-input-3-b672e48e23b2>:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(data.mean(), inplace=True)  # Fill NaN with the column mean


# Untuned

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import time

# Assuming the data is already loaded as 'data'
n_test = 30
n_lag = 100  # Rolling window size

# Function to create rolling windows
def create_rolling_window(series, window_size):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)

# Placeholder for results
results = []

# Set the same hyperparameters for all models
n_epochs = 100
batch_size = 32
n_neurons = 50
input_shape = (n_lag,)  # MLP input is (n_lag,)

# Scaling the data using MinMaxScaler (0 to 1 range)
scaler = MinMaxScaler()

# Loop over each column (excluding the 'Date' column)
for col in data.columns:
    print(f'Training model for column: {col}')

    # Prepare the data for the current column
    series = data[col].values.reshape(-1, 1)  # Reshape for scaling (1 feature)

    # Scale the series
    series_scaled = scaler.fit_transform(series)

    # Create rolling windows
    X, y = create_rolling_window(series_scaled, n_lag)

    # Split into train and test
    X_train, X_test = X[:-n_test], X[-n_test:]
    y_train, y_test = y[:-n_test], y[-n_test:]

    # For MLP, the input shape is (samples, features), no need for 3D reshape
    X_train = X_train.reshape((X_train.shape[0], n_lag))
    X_test = X_test.reshape((X_test.shape[0], n_lag))

    # Build the MLP model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(n_neurons, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    # Measure training time
    start_time = time.time()

    # Train the model
    model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0)

    # Record the time taken
    training_time = time.time() - start_time

    # Make predictions
    y_pred = model.predict(X_test)

    # Inverse transform the predicted and test data to original scale
    y_pred_inv = scaler.inverse_transform(y_pred)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate MSE and MAPE
    mse = mean_squared_error(y_test_inv, y_pred_inv)
    mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

    # Store the results
    results.append({
        'Column': col,
        'MSE': mse,
        'MAPE': mape,
        'Training Time (s)': training_time
    })

# Convert the results into a DataFrame and display it
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('ATM_MLP_univariate_scaled_without_tuning.csv', index=False)

# Optionally, download the file (if you're on Google Colab or similar)
from google.colab import files
files.download('ATM_MLP_univariate_scaled_without_tuning.csv')

Training model for column: NN5 - 001


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Training model for column: NN5 - 002


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Training model for column: NN5 - 003


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Training model for column: NN5 - 111


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Tuned

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import time
from itertools import product

# Assuming the data is already loaded as 'data'
n_test = 30
n_lag = 100  # Rolling window size

# Function to create rolling windows
def create_rolling_window(series, window_size):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)

# Placeholder for results
results = []

# Set the hyperparameters for tuning
hidden_layers_options = [1, 2]
neurons_options = [64, 128]
learning_rates = [0.1, 0.01, 0.001]
batch_sizes = [32, 64]
n_epochs = 100

# Scaling the data using MinMaxScaler (0 to 1 range)
scaler = MinMaxScaler()

data = data.iloc[:, 91:].copy()
# Loop over each column (excluding the 'Date' column)
for col in data.columns:
    print(f'Training model for column: {col}')

    # Prepare the data for the current column
    series = data[col].values.reshape(-1, 1)  # Reshape for scaling (1 feature)

    # Scale the series
    series_scaled = scaler.fit_transform(series)

    # Create rolling windows
    X, y = create_rolling_window(series_scaled, n_lag)

    # Split into train and test
    X_train, X_test = X[:-n_test], X[-n_test:]
    y_train, y_test = y[:-n_test], y[-n_test:]

    # Reshape the data for MLP [samples, features]
    X_train = X_train.reshape((X_train.shape[0], n_lag))
    X_test = X_test.reshape((X_test.shape[0], n_lag))

    # Iterate over all combinations of hyperparameters
    for num_layers, num_neurons, lr, batch_size in product(hidden_layers_options, neurons_options, learning_rates, batch_sizes):

        print(f"Hyperparameters: Layers: {num_layers}, Neurons: {num_neurons}, LR: {lr}, Batch Size: {batch_size}")

        # Build the MLP model
        model = tf.keras.models.Sequential()

        # Add hidden layers
        model.add(tf.keras.layers.Dense(num_neurons, activation='relu', input_shape=(n_lag,)))
        for i in range(1, num_layers):
            model.add(tf.keras.layers.Dense(num_neurons, activation='relu'))

        # Add the output layer
        model.add(tf.keras.layers.Dense(1))

        # Compile the model with the specified learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mse')

        # Measure training time
        start_time = time.time()

        # Train the model
        model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0)

        # Record the time taken
        training_time = time.time() - start_time

        # Make predictions
        y_pred = model.predict(X_test)

        # Inverse transform the predicted and test data to original scale
        y_pred_inv = scaler.inverse_transform(y_pred)
        y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Check for NaN values in the predictions or test data
        if np.isnan(y_pred_inv).any() or np.isnan(y_test_inv).any():
            print(f"NaN values detected in column {col} for the hyperparameters: Layers={num_layers}, Neurons={num_neurons}, LR={lr}, Batch Size={batch_size}")
            continue  # Skip this iteration if NaNs are present

        # Calculate MSE and MAPE
        mse = mean_squared_error(y_test_inv, y_pred_inv)
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

        # Store the results
        results.append({
            'Column': col,
            'Layers': num_layers,
            'Neurons': num_neurons,
            'Learning Rate': lr,
            'Batch Size': batch_size,
            'MSE': mse,
            'MAPE': mape,
            'Training Time (s)': training_time
        })

# Convert the results into a DataFrame and display it
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file for download
results_df.to_csv('ATM_MLP_univariate_scaled_with_tuning_111.csv', index=False)

# Optionally, download the file (if you're on Google Colab or similar)
from google.colab import files
files.download('ATM_MLP_univariate_scaled_with_tuning_111.csv')

Training model for column: NN5 - 092
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.1, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.1, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.01, Batch Size: 32


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Hyperparameters: Layers: 2, Neurons: 128, LR: 0.001, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install tslearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 20.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import StandardScaler

# Assuming the data is already loaded as 'data'
# Exclude the 'Date' column if present, keep only the series columns for clustering
time_series_data = data.copy()

# Normalize the data for clustering
scaler = StandardScaler()
normalized_data = scaler.fit_transform(time_series_data.T)  # Transpose to have series in rows

# Get the column names (time series identifiers)
column_names = time_series_data.columns.tolist()

# 1. Euclidean Distance Clustering (KMeans)
print("Performing KMeans clustering using Euclidean distance...")

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(normalized_data)
euclidean_labels = kmeans.labels_

# List of column names in each cluster (Euclidean)
euclidean_cluster_1 = [column_names[i] for i in np.where(euclidean_labels == 0)[0].tolist()]
euclidean_cluster_2 = [column_names[i] for i in np.where(euclidean_labels == 1)[0].tolist()]

print(f"Cluster 1 (Euclidean): {euclidean_cluster_1}")
print(f"Cluster 2 (Euclidean): {euclidean_cluster_2}")

# 2. DTW Clustering (TimeSeriesKMeans)
print("Performing DTW-based clustering...")

dtw_kmeans = TimeSeriesKMeans(n_clusters=2, metric="dtw", random_state=42)
dtw_kmeans.fit(time_series_data.T)  # Using transpose to cluster the series
dtw_labels = dtw_kmeans.labels_

# List of column names in each cluster (DTW)
dtw_cluster_1 = [column_names[i] for i in np.where(dtw_labels == 0)[0].tolist()]
dtw_cluster_2 = [column_names[i] for i in np.where(dtw_labels == 1)[0].tolist()]

print(f"Cluster 1 (DTW): {dtw_cluster_1}")
print(f"Cluster 2 (DTW): {dtw_cluster_2}")

Performing KMeans clustering using Euclidean distance...
Cluster 1 (Euclidean): ['NN5 - 002', 'NN5 - 006', 'NN5 - 007', 'NN5 - 009', 'NN5 - 010', 'NN5 - 012', 'NN5 - 013', 'NN5 - 014', 'NN5 - 015', 'NN5 - 020', 'NN5 - 021', 'NN5 - 022', 'NN5 - 023', 'NN5 - 025', 'NN5 - 026', 'NN5 - 027', 'NN5 - 028', 'NN5 - 031', 'NN5 - 033', 'NN5 - 035', 'NN5 - 036', 'NN5 - 037', 'NN5 - 039', 'NN5 - 040', 'NN5 - 041', 'NN5 - 044', 'NN5 - 045', 'NN5 - 046', 'NN5 - 049', 'NN5 - 051', 'NN5 - 053', 'NN5 - 054', 'NN5 - 055', 'NN5 - 056', 'NN5 - 057', 'NN5 - 059', 'NN5 - 060', 'NN5 - 061', 'NN5 - 063', 'NN5 - 064', 'NN5 - 069', 'NN5 - 070', 'NN5 - 071', 'NN5 - 072', 'NN5 - 073', 'NN5 - 074', 'NN5 - 075', 'NN5 - 076', 'NN5 - 077', 'NN5 - 078', 'NN5 - 079', 'NN5 - 081', 'NN5 - 082', 'NN5 - 084', 'NN5 - 085', 'NN5 - 086', 'NN5 - 088', 'NN5 - 089', 'NN5 - 091', 'NN5 - 092', 'NN5 - 094', 'NN5 - 097', 'NN5 - 099', 'NN5 - 100', 'NN5 - 103', 'NN5 - 105', 'NN5 - 106', 'NN5 - 107', 'NN5 - 108', 'NN5 - 111']
Cluster 2

In [ ]:
# Assuming you have already run the clustering and obtained 'euclidean_cluster_1', 'euclidean_cluster_2', 'dtw_cluster_1', and 'dtw_cluster_2'

# Convert the clusters into sets for easier comparison
euclidean_cluster_1_set = set(euclidean_cluster_1)
euclidean_cluster_2_set = set(euclidean_cluster_2)
dtw_cluster_1_set = set(dtw_cluster_1)
dtw_cluster_2_set = set(dtw_cluster_2)

# Compare the clusters: Items that are in both Euclidean Cluster 1 and DTW Cluster 1
common_in_cluster_1 = euclidean_cluster_1_set.intersection(dtw_cluster_1_set)
common_in_cluster_2 = euclidean_cluster_2_set.intersection(dtw_cluster_2_set)

# Compare the clusters: Items that are in Euclidean Cluster 1 and DTW Cluster 2 (and vice versa)
common_between_different_clusters_1 = euclidean_cluster_1_set.intersection(dtw_cluster_2_set)
common_between_different_clusters_2 = euclidean_cluster_2_set.intersection(dtw_cluster_1_set)

# Summary of comparison
print(f"Common items in Euclidean Cluster 1 and DTW Cluster 1: {len(common_in_cluster_1)}")
print(f"Common items in Euclidean Cluster 2 and DTW Cluster 2: {len(common_in_cluster_2)}")
print(f"Common items in Euclidean Cluster 1 and DTW Cluster 2: {len(common_between_different_clusters_1)}")
print(f"Common items in Euclidean Cluster 2 and DTW Cluster 1: {len(common_between_different_clusters_2)}")

Common items in Euclidean Cluster 1 and DTW Cluster 1: 65
Common items in Euclidean Cluster 2 and DTW Cluster 2: 40
Common items in Euclidean Cluster 1 and DTW Cluster 2: 5
Common items in Euclidean Cluster 2 and DTW Cluster 1: 1


# One random tuned model for whole cluster

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import random
from itertools import product

# Function to create rolling windows
def create_rolling_window(series, window_size):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)

# Scaling function
def scale_data(series):
    scaler = MinMaxScaler()
    return scaler, scaler.fit_transform(series.reshape(-1, 1))

# MLP Training Function with Hyperparameter Tuning
def train_mlp_model_with_tuning(X_train, y_train, X_val, y_val, n_lag, hidden_layers_options, neurons_options, learning_rates, batch_sizes, n_epochs=100):
    best_model = None
    best_mse = float('inf')
    best_hyperparams = None

    # Flatten the input for MLP as it expects 2D input [samples, features]
    X_train_flat = X_train.reshape(X_train.shape[0], n_lag)
    X_val_flat = X_val.reshape(X_val.shape[0], n_lag)

    # Iterate over all combinations of hyperparameters
    for num_layers, num_neurons, lr, batch_size in product(hidden_layers_options, neurons_options, learning_rates, batch_sizes):

        print(f"Hyperparameters: Layers: {num_layers}, Neurons: {num_neurons}, LR: {lr}, Batch Size: {batch_size}")

        # Build the MLP model
        model = tf.keras.models.Sequential()

        # Add hidden layers
        for _ in range(num_layers):
            model.add(tf.keras.layers.Dense(num_neurons, activation='relu', input_dim=n_lag))

        # Add the output layer
        model.add(tf.keras.layers.Dense(1))

        # Compile the model with the specified learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mse')

        # Train the model
        model.fit(X_train_flat, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0)

        # Validate the model to check performance
        y_val_pred = model.predict(X_val_flat)
        mse_val = mean_squared_error(y_val, y_val_pred)

        # Keep track of the best model and hyperparameters
        if mse_val < best_mse:
            best_mse = mse_val
            best_model = model
            best_hyperparams = (num_layers, num_neurons, lr, batch_size)

    # Return the best trained model and best hyperparameters
    return best_model, best_hyperparams

# Function to process and predict a cluster
def process_cluster_with_tuning(cluster_data, cluster_name, n_lag=100, n_test=30, hidden_layers_options=[1, 2], neurons_options=[64, 128], learning_rates=[0.1, 0.01, 0.001], batch_sizes=[32, 64], n_epochs=100):
    # Select a random series from the cluster
    selected_series = random.choice(cluster_data.columns)

    # Print the name of the randomly selected time series
    print(f"Randomly selected time series for {cluster_name}: {selected_series}")

    # Prepare the data for the selected series
    series = cluster_data[selected_series].values
    scaler, series_scaled = scale_data(series)

    # Create rolling windows
    X, y = create_rolling_window(series_scaled, n_lag)

    # Split into train, test, and validation
    X_train, X_val, X_test = X[:-n_test*2], X[-n_test*2:-n_test], X[-n_test:]
    y_train, y_val, y_test = y[:-n_test*2], y[-n_test*2:-n_test], y[-n_test:]

    # Train the MLP model with hyperparameter tuning
    best_model, best_hyperparams = train_mlp_model_with_tuning(X_train, y_train, X_val, y_val, n_lag, hidden_layers_options, neurons_options, learning_rates, batch_sizes, n_epochs)

    # Predict the rest of the series in the cluster using the best model
    results = []
    for col in cluster_data.columns:
        series_to_predict = cluster_data[col].values
        _, series_scaled_to_predict = scale_data(series_to_predict)
        X_to_predict, y_to_predict = create_rolling_window(series_scaled_to_predict, n_lag)

        # Flatten for MLP input
        X_to_predict_flat = X_to_predict.reshape(X_to_predict.shape[0], n_lag)

        # Predict using the best model
        y_pred = best_model.predict(X_to_predict_flat)

        # Inverse transform the predicted and test data to original scale
        y_pred_inv = scaler.inverse_transform(y_pred)
        y_test_inv = scaler.inverse_transform(y_to_predict.reshape(-1, 1))

        # Calculate MSE and MAPE
        mse = mean_squared_error(y_test_inv, y_pred_inv)
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

        # Store the results in a list of dictionaries
        results.append({
            'Cluster': cluster_name,
            'Time Series': col,
            'Selected Series for Training': selected_series,
            'MSE': mse,
            'MAPE': mape,
            'Best Hyperparameters': best_hyperparams
        })

    return pd.DataFrame(results)

# Assuming you have already created clusters (euclidean_cluster_1, euclidean_cluster_2, dtw_cluster_1, dtw_cluster_2)
# Process each cluster and train an MLP model for one series in each

# For Euclidean Cluster 1
euclidean_cluster_1_results = process_cluster_with_tuning(data[euclidean_cluster_1], "Euclidean Cluster 1")

# For Euclidean Cluster 2
euclidean_cluster_2_results = process_cluster_with_tuning(data[euclidean_cluster_2], "Euclidean Cluster 2")

# For DTW Cluster 1
dtw_cluster_1_results = process_cluster_with_tuning(data[dtw_cluster_1], "DTW Cluster 1")

# For DTW Cluster 2
dtw_cluster_2_results = process_cluster_with_tuning(data[dtw_cluster_2], "DTW Cluster 2")

# Combine all results into one DataFrame
all_results = pd.concat([euclidean_cluster_1_results, euclidean_cluster_2_results, dtw_cluster_1_results, dtw_cluster_2_results])

# # Save the combined results to a CSV file
# all_results.to_csv('clustered_mlp_results.csv', index=False)

# # Show the results
# print(all_results)

# # Optionally, download the file (if you're using Google Colab or similar)
# from google.colab import files
# files.download('clustered_mlp_results.csv')


Randomly selected time series for Euclidean Cluster 1: NN5 - 070
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.1, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.1, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.01, Batch Size: 32


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Hyperparameters: Layers: 2, Neurons: 128, LR: 0.001, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Hyperparameters: Layers: 2, Neurons: 128, LR: 0.001, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22

# multivarite

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import random
from itertools import product
import time

# Function to create rolling windows for multivariate input
def create_multivariate_rolling_window(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size].flatten())  # Flatten to convert to MLP input shape
        y.append(data[i + window_size, target_index])
    return np.array(X), np.array(y)

# Scaling function for multivariate data
def scale_multivariate_data(data):
    scaler = MinMaxScaler()
    return scaler, scaler.fit_transform(data)

# Multivariate MLP Training Function with Hyperparameter Tuning
def train_multivariate_mlp_with_tuning(X_train, y_train, X_val, y_val, hidden_layers_options, neurons_options, learning_rates, batch_sizes, n_epochs=100):
    best_model = None
    best_mse = float('inf')
    best_hyperparams = None
    training_times_per_setting = []

    # Iterate over all combinations of hyperparameters
    for num_layers, num_neurons, lr, batch_size in product(hidden_layers_options, neurons_options, learning_rates, batch_sizes):
        print(f"Hyperparameters: Layers: {num_layers}, Neurons: {num_neurons}, LR: {lr}, Batch Size: {batch_size}")

        # Build the MLP model
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)))

        # Add hidden layers
        for _ in range(num_layers):
            model.add(tf.keras.layers.Dense(num_neurons, activation='relu'))

        # Add the output layer
        model.add(tf.keras.layers.Dense(1))

        # Compile the model with the specified learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mse')

        # Track the start time
        start_time = time.time()

        # Train the model
        model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0)

        # Track the end time and calculate training duration
        end_time = time.time()
        training_duration = end_time - start_time
        training_times_per_setting.append((num_layers, num_neurons, lr, batch_size, training_duration))

        print(f"Training time for current setting: {training_duration:.2f} seconds")

        # Validate the model to check performance
        y_val_pred = model.predict(X_val)
        mse_val = mean_squared_error(y_val, y_val_pred)

        # Keep track of the best model and hyperparameters
        if mse_val < best_mse:
            best_mse = mse_val
            best_model = model
            best_hyperparams = (num_layers, num_neurons, lr, batch_size)

    # Return the best trained model, best hyperparameters, and individual training times
    return best_model, best_hyperparams, training_times_per_setting

# Function to process a cluster, tune on one series, and apply the model to all series
def process_cluster_with_tuning(cluster_data, cluster_name, n_lag=100, n_test=30, hidden_layers_options=[1, 2], neurons_options=[64, 128], learning_rates=[0.1, 0.01, 0.001], batch_sizes=[32, 64], n_epochs=100):
    results = []

    # Select a random series from the cluster for tuning
    selected_series = random.choice(cluster_data.columns)
    target_index = cluster_data.columns.get_loc(selected_series)
    print(f"Tuning model for target series {selected_series} in cluster {cluster_name}")

    # Scale the data for all series in the cluster
    scaler, scaled_data = scale_multivariate_data(cluster_data.values)

    # Create rolling windows with multivariate data, using selected series as target
    X, y = create_multivariate_rolling_window(scaled_data, n_lag, target_index)

    # Split into train, validation, and test sets
    X_train, X_val, X_test = X[:-n_test*2], X[-n_test*2:-n_test], X[-n_test:]
    y_train, y_val, y_test = y[:-n_test*2], y[-n_test*2:-n_test], y[-n_test:]

    # Train the multivariate MLP model with hyperparameter tuning on the selected series
    best_model, best_hyperparams, training_times_per_setting = train_multivariate_mlp_with_tuning(X_train, y_train, X_val, y_val, hidden_layers_options, neurons_options, learning_rates, batch_sizes, n_epochs)

    # Display individual training times per setting
    for (num_layers, num_neurons, lr, batch_size, duration) in training_times_per_setting:
        print(f"Hyperparameters: Layers={num_layers}, Neurons={num_neurons}, LR={lr}, Batch Size={batch_size} -> Training Time: {duration:.2f} seconds")

    # Apply the tuned model to forecast each series in the cluster
    for target_series in cluster_data.columns:
        target_index = cluster_data.columns.get_loc(target_series)
        print(f"Applying tuned model to forecast series {target_series} in cluster {cluster_name}")

        # Create rolling windows for the current target series
        X_all, y_all = create_multivariate_rolling_window(scaled_data, n_lag, target_index)

        # Use the test portion
        X_test, y_test = X_all[-n_test:], y_all[-n_test:]

        # Predict using the tuned model
        y_pred = best_model.predict(X_test)

        # Inverse transform the predicted and test data to the original scale for the target series
        y_pred_full = np.zeros((y_pred.shape[0], scaled_data.shape[1]))  # Zero-padding for inverse transform
        y_pred_full[:, target_index] = y_pred.flatten()
        y_pred_inv = scaler.inverse_transform(y_pred_full)[:, target_index]

        y_test_full = np.zeros((y_test.shape[0], scaled_data.shape[1]))
        y_test_full[:, target_index] = y_test.flatten()
        y_test_inv = scaler.inverse_transform(y_test_full)[:, target_index]

        # Calculate MSE and MAPE for the target series
        mse = mean_squared_error(y_test_inv, y_pred_inv)
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

        # Append the results
        results.append({
            'Cluster': cluster_name,
            'Time Series': target_series,
            'MSE': mse,
            'MAPE': mape,
            'Best Hyperparameters': best_hyperparams,
            'Training Times (s) per Setting': training_times_per_setting
        })

    return pd.DataFrame(results)

# For Euclidean Cluster 1
euclidean_cluster_1_results = process_cluster_with_tuning(data[euclidean_cluster_1], "Euclidean Cluster 1")

# For Euclidean Cluster 2
euclidean_cluster_2_results = process_cluster_with_tuning(data[euclidean_cluster_2], "Euclidean Cluster 2")

# For DTW Cluster 1
dtw_cluster_1_results = process_cluster_with_tuning(data[dtw_cluster_1], "DTW Cluster 1")

# For DTW Cluster 2
dtw_cluster_2_results = process_cluster_with_tuning(data[dtw_cluster_2], "DTW Cluster 2")

# Combine all results into one DataFrame
all_results = pd.concat([euclidean_cluster_1_results, euclidean_cluster_2_results, dtw_cluster_1_results, dtw_cluster_2_results])

# Save or display results
print(all_results)


Tuning model for target series NN5 - 025 in cluster Euclidean Cluster 1
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.1, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training time for current setting: 5.92 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.1, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training time for current setting: 4.77 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.01, Batch Size: 32


...

Training time for current setting: 4.49 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Hyperparameters: Layers: 2, Neurons: 128, LR: 0.001, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training time for current setting: 5.82 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Hyperparameters: Layers: 2, Neurons: 128, LR: 0.001, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training time for current setting: 4.74 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Hyperparameters: Layers=1, Neurons=64, LR=0.1, Batch Size=32 -> Training Time: 5.18 seconds
Hyperparameters: Layers=1, Neurons=64, LR=0.1, Batch Size=64 -> Training Time: 4.24 seconds
Hyperparameters: Layers=1, Neurons=64, LR=0.01, Batch Size=32 -> Training Time: 5.26 seconds
Hyperparameters: Layers=1, Neurons=64, LR=0.01, Batch Size=64 -> Training Time: 4.33 seconds
Hyperparameters: Layers=1, Neurons=64, LR=0.001, Batch Size=32 -> Training Time: 5.26 seconds
Hyperparameters: Layers=1, Neurons=64, LR=0.001, Batch Size=64 -> Training Time: 4.01 seconds
Hyperparameters: Layers=1, Neurons=128, LR=0.1, Batch Size=32 -> Training Time: 5.49 seconds
Hyperparameters: Layers=1, Neurons=128, LR=0.1, Batch Size=64 -> Training Time: 4.10 seconds
Hyperparameters: Layers=1, Neurons=128, LR=0.01, Batch Size=32 -> Training Time: 5.19 seconds
Hyperparameters: Layers=1, Neurons=128, LR=0.01, Batch Size=64 -> Training 

# Create the average of a cluster and then tune it and apply it for whole cluster

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import random
from itertools import product
import time

# Function to create input and output data for MLP
def create_mlp_data(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

# Scaling function for univariate data
def scale_univariate_data(data):
    scaler = MinMaxScaler()
    # Handle NaN values by filling them with the mean of the data
    data = np.nan_to_num(data, nan=np.nanmean(data))
    return scaler, scaler.fit_transform(data.reshape(-1, 1))

# MLP Training Function with Hyperparameter Tuning
def train_mlp_with_tuning(X_train, y_train, X_val, y_val, hidden_layers_options, neurons_options, learning_rates, batch_sizes, n_epochs=100):
    best_model = None
    best_mse = float('inf')
    best_hyperparams = None
    training_times_per_setting = []

    # Iterate over all combinations of hyperparameters
    for num_layers, num_neurons, lr, batch_size in product(hidden_layers_options, neurons_options, learning_rates, batch_sizes):
        print(f"Hyperparameters: Layers: {num_layers}, Neurons: {num_neurons}, LR: {lr}, Batch Size: {batch_size}")

        # Build the MLP model
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Flatten(input_shape=(X_train.shape[1],)))

        # Add hidden layers
        for _ in range(num_layers):
            model.add(tf.keras.layers.Dense(num_neurons, activation='relu'))

        # Add the output layer
        model.add(tf.keras.layers.Dense(1))

        # Compile the model with the specified learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mse')

        # Track the start time
        start_time = time.time()

        # Train the model
        model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0)

        # Track the end time and calculate training duration
        end_time = time.time()
        training_duration = end_time - start_time
        training_times_per_setting.append((num_layers, num_neurons, lr, batch_size, training_duration))

        print(f"Training time for current setting: {training_duration:.2f} seconds")

        # Validate the model to check performance
        y_val_pred = model.predict(X_val)
        mse_val = mean_squared_error(y_val, y_val_pred)

        # Keep track of the best model and hyperparameters
        if mse_val < best_mse:
            best_mse = mse_val
            best_model = model
            best_hyperparams = (num_layers, num_neurons, lr, batch_size)

    # Return the best trained model, best hyperparameters, and individual training times
    return best_model, best_hyperparams, training_times_per_setting

# Function to process a cluster, tune on average series, and apply the model to each series in the cluster
def process_cluster_with_tuning(cluster_data, cluster_name, n_lag=100, n_test=30, hidden_layers_options=[1, 2], neurons_options=[64, 128], learning_rates=[0.1, 0.01, 0.001], batch_sizes=[32, 64], n_epochs=100):
    results = []

    # Calculate the average series for the cluster
    avg_series = cluster_data.mean(axis=1).values
    print(f"Tuning model for average series in cluster {cluster_name}")

    # Scale the average series data
    scaler, scaled_avg_series = scale_univariate_data(avg_series)

    # Create data for MLP using sliding window approach
    X, y = create_mlp_data(scaled_avg_series, n_lag)

    # Split into train, validation, and test sets
    X_train, X_val, X_test = X[:-n_test*2], X[-n_test*2:-n_test], X[-n_test:]
    y_train, y_val, y_test = y[:-n_test*2], y[-n_test*2:-n_test], y[-n_test:]

    # Train the MLP model with hyperparameter tuning on the average series
    best_model, best_hyperparams, training_times_per_setting = train_mlp_with_tuning(X_train, y_train, X_val, y_val, hidden_layers_options, neurons_options, learning_rates, batch_sizes, n_epochs)

    # Display individual training times per setting
    for (num_layers, num_neurons, lr, batch_size, duration) in training_times_per_setting:
        print(f"Hyperparameters: Layers={num_layers}, Neurons={num_neurons}, LR={lr}, Batch Size={batch_size} -> Training Time: {duration:.2f} seconds")

    # Apply the tuned model to forecast each series in the cluster
    for target_series in cluster_data.columns:
        print(f"Applying tuned model to forecast series {target_series} in cluster {cluster_name}")

        # Scale each target series data
        series_data = cluster_data[target_series].values
        # Fill any NaN values in the target series with the mean of the series
        series_data = np.nan_to_num(series_data, nan=np.nanmean(series_data))
        _, scaled_series = scale_univariate_data(series_data)

        # Create data for MLP using sliding window approach
        X_all, y_all = create_mlp_data(scaled_series, n_lag)

        # Use the test portion
        X_test, y_test = X_all[-n_test:], y_all[-n_test:]

        # Predict using the tuned model
        y_pred = best_model.predict(X_test)

        # Inverse transform the predicted and test data to the original scale
        y_pred_inv = scaler.inverse_transform(y_pred)
        y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Calculate MSE and MAPE for the target series
        mse = mean_squared_error(y_test_inv, y_pred_inv)
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

        # Append the results
        results.append({
            'Cluster': cluster_name,
            'Time Series': target_series,
            'MSE': mse,
            'MAPE': mape,
            'Best Hyperparameters': best_hyperparams,
            'Training Times (s) per Setting': training_times_per_setting
        })

    return pd.DataFrame(results)

# Assuming data, euclidean_cluster_1, euclidean_cluster_2, dtw_cluster_1, dtw_cluster_2 are predefined
# Example usage for clusters
euclidean_cluster_1_results = process_cluster_with_tuning(data[euclidean_cluster_1], "Euclidean Cluster 1")
euclidean_cluster_2_results = process_cluster_with_tuning(data[euclidean_cluster_2], "Euclidean Cluster 2")
dtw_cluster_1_results = process_cluster_with_tuning(data[dtw_cluster_1], "DTW Cluster 1")
dtw_cluster_2_results = process_cluster_with_tuning(data[dtw_cluster_2], "DTW Cluster 2")

# Combine all results into one DataFrame
all_results = pd.concat([euclidean_cluster_1_results, euclidean_cluster_2_results, dtw_cluster_1_results, dtw_cluster_2_results])

# Save or display results
print(all_results)


Tuning model for average series in cluster Euclidean Cluster 1
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.1, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training time for current setting: 6.16 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.1, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training time for current setting: 4.28 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Hyperparameters: Layers: 1, Neurons: 64, LR: 0.01, Batch Size: 32


...

Training time for current setting: 4.44 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Hyperparameters: Layers: 2, Neurons: 128, LR: 0.001, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training time for current setting: 6.98 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Hyperparameters: Layers: 2, Neurons: 128, LR: 0.001, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training time for current setting: 4.56 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Hyperparameters: Layers=1, Neurons=64, LR=0.1, Batch Size=32 -> Training Time: 5.13 seconds
Hyperparameters: Layers=1, Neurons=64, LR=0.1, Batch Size=64 -> Training Time: 4.01 seconds
Hyperparameters: Layers=1, Neurons=64, LR=0.01, Batch Size=32 -> Training Time: 5.08 seconds
Hyperparameters: Layers=1, Neurons=64, LR=0.01, Batch Size=64 -> Training Time: 4.16 seconds
Hyperparameters: Layers=1, Neurons=64, LR=0.001, Batch Size=32 -> Training Time: 5.08 seconds
Hyperparameters: Layers=1, Neurons=64, LR=0.001, Batch Size=64 -> Training Time: 4.02 seconds
Hyperparameters: Layers=1, Neurons=128, LR=0.1, Batch Size=32 -> Training Time: 5.20 seconds
Hyperparameters: Layers=1, Neurons=128, LR=0.1, Batch Size=64 -> Training Time: 4.02 seconds
Hyperparameters: Layers=1, Neurons=128, LR=0.01, Batch Size=32 -> Training Time: 5.22 seconds
Hyperparameters: Layers=1, Neurons=128, LR=0.01, Batch Size=64 -> Training 